##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TensorBoard in Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorBoard can be used directly within notebook experiences such as [Colab](https://colab.research.google.com/) and [Jupyter](https://jupyter.org/). This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

## Setup

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

**For Jupyter users:** If you’ve installed Jupyter and TensorBoard into
the same virtualenv, then you should be good to go. If you’re using a
more complicated setup, like a global Jupyter installation and kernels
for different Conda/virtualenv environments, then you must ensure that
the `tensorboard` binary is on your `PATH` inside the Jupyter notebook
context. One way to do this is to modify the `kernel_spec` to prepend
the environment’s `bin` directory to `PATH`, [as described here][1].

[1]: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997


**For Docker users:** In case you are running a [Docker](https://docs.docker.com/install/) image of [Jupyter Notebook server using TensorFlow's nightly](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), it is necessary to expose not only the notebook's port, but the TensorBoard's port. Thus, run the container with the following command:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter 
```

where the `-p 6006` is the default port of TensorBoard. This will allocate a port for you to run one TensorBoard instance. To have concurrent instances, it is necessary to allocate more ports. Also, pass `--bind_all` to `%tensorboard` to expose the port outside the container.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Import TensorFlow, datetime, and os:

In [2]:
import tensorflow as tf
import datetime, os

## TensorBoard in notebooks

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Create a very simple model:

In [4]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Train the model using Keras and the TensorBoard callback:

In [5]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 0.4963 - accuracy: 0.8219 - val_loss: 0.4221 - val_accuracy: 0.8517
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3843 - accuracy: 0.8598 - val_loss: 0.3792 - val_accuracy: 0.8611
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3510 - accuracy: 0.8711 - val_loss: 0.3635 - val_accuracy: 0.8689
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3273 - accuracy: 0.8788 - val_loss: 0.3819 - val_accuracy: 0.8615
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3114 - accuracy: 0.8834 - val_loss: 0.3717 - val_accuracy: 0.8674


Start TensorBoard within the notebook using [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [6]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 31344), started 13:23:40 ago. (Use '!kill 31344' to kill it.)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

You can now view dashboards such as scalars, graphs, histograms, and others. Some dashboards are not available yet in Colab (such as the profile plugin).

The `%tensorboard` magic has exactly the same format as the TensorBoard command line invocation, but with a `%`-sign in front of it.

You can also start TensorBoard before training to monitor it in progress:

In [7]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 31344), started 13:24:21 ago. (Use '!kill 31344' to kill it.)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

The same TensorBoard backend is reused by issuing the same command. If a different logs directory was chosen, a new instance of TensorBoard would be opened. Ports are managed automatically. 

Start training a new model and watch TensorBoard update automatically every 30 seconds or refresh it with the button on the top right:

In [8]:
train_model()

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4958 - accuracy: 0.8218 - val_loss: 0.4071 - val_accuracy: 0.8556
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3838 - accuracy: 0.8605 - val_loss: 0.3781 - val_accuracy: 0.8608
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3494 - accuracy: 0.8716 - val_loss: 0.3828 - val_accuracy: 0.8605
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3282 - accuracy: 0.8799 - val_loss: 0.3718 - val_accuracy: 0.8662
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3132 - accuracy: 0.8839 - val_loss: 0.3608 - val_accuracy: 0.8697


You can use the `tensorboard.notebook` APIs for a bit more control:

In [9]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs/fit (started 17 days, 13:59:09 ago; pid 17248)
  - port 6006: logdir logs (started 13:25:00 ago; pid 31344)


In [10]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir logs (started 13:25:04 ago; port 6006, pid 31344).


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->